In [ ]:
import numpy as np
import pandas as pd
#делаем из json фрейм
import json 
from pandas.io.json import json_normalize #нормализуем json
from sklearn import preprocessing
import warnings
warnings.simplefilter('ignore', FutureWarning)
import seaborn as sns

In [ ]:
def cut_for_speed(obj):
    msk = np.random.rand(len(obj)) <0.1 #10% строк для ускорения
    nobj = obj[msk]
    return nobj

In [ ]:
def json_convert(df,json_columns): #преобразование json
    for column in  json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

In [ ]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
import csv
data = pd.read_csv("../input/ga-customer-revenue-prediction/train.csv", sep = ",", dtype = {'fullVisitorId': 'str'},
                   converters={column: json.loads for column in JSON_COLUMNS}, 
                nrows = None).sample(frac=0.3)
data = json_convert(data, JSON_COLUMNS) #конвертирование json
#отчистка мусора
delete = data.ne ('not available in demo dataset')
data = data.loc[delete.any(1), delete.any()]

In [ ]:
data.info()

In [ ]:
print(data.shape)


In [ ]:
def prepare_data(df):

    df = df.replace({'socialEngagementType': {'Engaged': True, 'Not Socially Engaged': False}})
    float_cols = ['totals.pageviews','totals.bounces','totals.transactionRevenue']
    for col in float_cols: 
        df[col].fillna(0, inplace=True)
        df[col] = df[col].astype(float)
    df['totals.transactionRevenue'] = np.log1p(df['totals.transactionRevenue'].values)
    str_cols = ['device.browser','geoNetwork.subContinent','geoNetwork.region',
            'geoNetwork.networkDomain','trafficSource.medium'] #считаем логорифм
    for col in str_cols:
        df = df.replace({col: {'not available in demo dataset': '(not set)'}})

        le = preprocessing.LabelEncoder()
        le.fit(df[col].unique())
        df[col] = le.transform(list(df[col].values.astype('str')))
        df[col] = df[col].astype(float)

    for columns in df.columns:
        if df[columns].dtype == 'object' or df[columns].dtype == 'int64':
            df = df.drop([columns], axis = 1)
    return df

In [ ]:
print(data.shape)


In [ ]:
data = prepare_data(data)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

y = data['totals.transactionRevenue'] #отделяем от выборки значения зависимой переменной
X = data.drop(columns=['totals.transactionRevenue'], axis=1) 
Xtrn, Xtest, Ytrn, Ytest = train_test_split(X, y, test_size = 0.2) #разбиваем выборку на тестовую и обуч.
models = [GradientBoostingRegressor(), #градиентный бустинг
RandomForestRegressor(), #случайный лес
SVR(kernel='linear'), #эпсилон-опорная векторная регрессия 
LinearRegression()] #линейная регрессия


In [ ]:
Result = pd.DataFrame() 
tmp = pd.DataFrame() 

for model in models: 
    m = str(model) 
    model_name = m[:m.index('(')] 
    scores = np.array([]) #кросс валидация
    fold = KFold(n_splits=15, shuffle=True) 
    scores_on_this_split = cross_val_score( 
        estimator = model, X=X, y=y, 
        cv=fold, scoring='r2') 
    scores = np.append(scores, scores_on_this_split) 
    print('Отработала', model_name) 
    tmp['Оценка качества'] = pd.Series(scores) 
    tmp['Модель'] = model_name 
    Result = Result.append([tmp]) #записывам данные

In [ ]:
Result

In [ ]:
#ящик с усами
Result = Result.replace({'Модель': {'GradientBoostingRegressor': 'Бустинг', 'RandomForestRegressor': 'Лес', 'LinearRegression': 'Линейная'}}) 
sns.boxplot(y = 'Оценка качества', x = 'Модель', data = Result)